In [ ]:
import os
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def sendNotification(app_name, message, additional_message='', event_name='program_log'):
    my_key = 'dtFZmutEg9ANUtSEpAU7Tu'
    data = {
        "value1" : app_name,
        "value2" : message,
        "value3" : additional_message
    }
    url = "https://maker.ifttt.com/trigger/" + event_name + "/with/key/" + my_key
    
    r = requests.post(url, data=data)
    
    return r

In [ ]:
def waitForElement(driver, by_what=By.XPATH, element_info='', delay=20, do_quit=True):
    try:
        elem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((by_what, element_info)))
        print("Page is ready!")
        return elem
    except TimeoutException:
        print("Loading took too much time!")
        print("quiting while waiting for element:", element_info)
        if do_quit:
            report = "quiting while waiting for element: " + element_info
            send_report_and_close(report, driver)
            # driver.quit()
        return None

In [ ]:
# sendNotification('Hello', 'Hello', event_name='program_log')

In [ ]:
def send_report_and_close(report, driver):
    for i in range(10):
        r = sendNotification('Jupyter DHR', report)
        if r.status_code == 200:
            break
        else:
            print("Hasn't send retrying... " + str(i) + " of 10")
  
    driver.close()
    exit()

In [ ]:
report = ''
url = 'https://xmuxg.xmu.edu.cn/app/214'
path_to_driver = os.path.join(os.path.curdir, 'chromedriver')

driver = webdriver.Chrome(path_to_driver)
driver.get(url)


In [ ]:
loaded_url = driver.current_url
if 'login' in loaded_url:
    # log in here then
    print('waiting to click')
    loging_button = waitForElement(driver, element_info="//button[contains(.,'统一身份认证')]")

    loging_button.click()


In [ ]:
    login_field = driver.find_element_by_xpath("//input[@id='username']")
    login_field.click()
    print('clicking login')
    # login_field.clear()
    login_field.send_keys('')

    password_field = driver.find_element_by_xpath("//input[@id='password']")
    password_field.click()
    print('clicking password')
    # password_field.clear()
    password_field.send_keys('')

In [ ]:
    # press login/登录
    loging_button = driver.find_element_by_xpath("//button[contains(.,'登录/Login')]")
    loging_button.click()
    
    if loaded_url != driver.current_url:
        print("logged in successfully")
        report += 'Login OK.'
    else:
        print("Hasn't logged in")
        report = "Hasn't logged in. Check log:pass"
        send_report_and_close(report, driver)
        # TODO: send_report_and_close()

#
# /html/body/div[1]/div[@id='main']/div[@class='page-container container-fluid']/div[@class='page-content row']/div[@class='col-sm-12 page left-show']/div[@id='mainPage-page']/div[@class='main-p']/div[@class='shadow_box box_wrap_2']/div[2]/div[@class='box_main box_flex']/div[@class='app_child box_flex'][1]/div[@class='grow_1 box_flex column justify_center']/div[@class='title box_flex']

In [ ]:
print("closing nav bar")
nav_bar = waitForElement(driver, element_info="//div[@class='menu-toggle pull-left']")
nav_bar.click()

In [ ]:
dhr_section = waitForElement(driver, element_info="//div[contains(text(),'Daily Health Report')]")

In [ ]:
dhr_section.click()

In [ ]:
# 健康打卡
menu_button = waitForElement(driver, element_info="//div[contains(@class, 'tab')][2]")

In [ ]:
# CHECK DATE
from datetime import datetime

curdate = datetime.today().strftime('%Y-%m-%d')

print(curdate)

In [ ]:
if len(driver.find_elements_by_xpath("//span[text()[contains(.,'" + curdate + "')]]")) > 0:
    print("date is correct:", curdate)
    report += "Date OK."
else:
    print("there's something wrong with date "+ curdate +", but we'll continue anyway")
    report += "Date BAD."

In [ ]:
# CHECK TIME
hours = int(datetime.today().strftime('%H'))
minutes = int(datetime.today().strftime('%M'))

if hours >= 16 and minutes >= 30:
    print("too late for the daily health report")
    report += "Time BAD."
    send_report_and_close(report, driver)
    # TODO: send_report_and_close()
else:
    print("time's alright:", hours, ':', minutes)
    report += "Time OK."

In [ ]:
# Finally getting to the reporting part,
# menu_button = driver.find_element_by_xpath("//div[contains(@class, 'tab')][2]")
# menu_button = driver.find_element_by_xpath("//div[text()='我的菜单')]")
menu_button.click()


In [ ]:
confirmation_field = driver.find_elements_by_xpath("//div[contains(@class, 'v-select btn-block info-value')]")[-1]# confirmation = driver.find_elements_by_xpath("//div[contains(@class, 'form-control dropdown-toggle')]")

In [ ]:
# confirmation

In [ ]:
if len(driver.find_elements_by_xpath("//span[text()[contains(.,'是 Yes')]]")) > 0:
    print("confirmation is already yes, consider checking website yourself")
    report += 'Yes already.'
    # clicking outside
    # driver.find_element_by_xpath("//body").click()
else:
    # hoping that the last element is the confirmation one
    # WebElement element = driver.findElement(By.id("id_of_element"));
    driver.execute_script("arguments[0].scrollIntoView(true)", confirmation)
# ((JavascriptExecutor) driver).executeScript("arguments[0].scrollIntoView(true);", element);
    confirmation.click()
    # yes_button = driver.find_element_by_xpath("//span[text()[contains(.,'是 Yes')]]/ancestor::label[@class='btn-block']")
    # yes_button.click()

In [ ]:
save_button = driver.find_elements_by_xpath("//span[text()[contains(.,'保存')]]")[-1]

In [ ]:
saved_suc = None
retry = -1
driver.execute_script("arguments[0].scrollIntoView(true);", save_button)
while saved_suc is None and retry < 10:
    retry += 1
    if retry > 0:
        print("Hasn't saved retrying:", retry, "of 10")
    driver.execute_script("arguments[0].click();", save_button)
    # save_button.click()
    print("4) clicked save")

    '''
        POPUP ALERT
    '''
    driver.switch_to.alert.accept()
    print("5) clicked OK on an alert window")

    '''
        保存成功
    '''
    saved_suc = waitForElement(driver, delay=7, element_info="//pre[contains(@class, 'message')]", do_quit=False)

report += 'Saved OK.'
if retry > 0:
    report += str(retry) + ' retry'

print("6) has been saved")

In [ ]:
# sendNotification("DH", report)
# driver.close()
print("It should be done by now")
send_report_and_close(report, driver)